# Benchmark: Parquet vs DuckDB for Price Loading

This notebook analyzes whether replacing DuckDB with direct PyArrow Parquet loading
would improve performance for `load_all_prices()` and `load_prices_matrix()`.

## Hypothesis

| Scenario | Expected Winner | Why |
|----------|-----------------|-----|
| Full file load (no date filter) | PyArrow | No SQL parse overhead, zero-copy columns |
| Date-filtered load (small range) | DuckDB | Pushdown predicate to Parquet row groups |
| Date-filtered load (most data) | PyArrow | Filter overhead < DuckDB overhead |

In [1]:
import time
import os
import numpy as np
import duckdb
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
from datetime import date

# Find the prices parquet file
PRICES_FILE = "../data/prices.parquet"
if not os.path.exists(PRICES_FILE):
    # Try alternate location
    PRICES_FILE = "data/prices.parquet"
    
print(f"Using prices file: {PRICES_FILE}")
print(f"File exists: {os.path.exists(PRICES_FILE)}")
if os.path.exists(PRICES_FILE):
    print(f"File size: {os.path.getsize(PRICES_FILE) / 1024 / 1024:.1f} MB")

Using prices file: ../data/prices.parquet
File exists: True
File size: 34.7 MB


## Step 1: Measure Current DuckDB Performance

In [2]:
from specparser.amt import load_all_prices, load_prices_matrix, clear_prices_dict, clear_prices_matrix

# Warm up (first call may include import overhead)
clear_prices_dict()
_ = load_all_prices(PRICES_FILE)
clear_prices_dict()

In [3]:
# Benchmark load_all_prices (no filter)
clear_prices_dict()
t0 = time.perf_counter()
prices_dict = load_all_prices(PRICES_FILE)
t1 = time.perf_counter()
duckdb_load_all_no_filter = t1 - t0
n_keys = len(prices_dict)
print(f"load_all_prices (no filter): {duckdb_load_all_no_filter:.3f}s, {n_keys:,} keys")

load_all_prices (no filter): 8.921s, 8,489,913 keys


In [4]:
# Benchmark load_all_prices (with 3-year date filter)
clear_prices_dict()
t0 = time.perf_counter()
prices_dict_filtered = load_all_prices(PRICES_FILE, "2022-01-01", "2024-12-31")
t1 = time.perf_counter()
duckdb_load_all_3yr_filter = t1 - t0
n_keys_filtered = len(prices_dict_filtered)
print(f"load_all_prices (3yr filter): {duckdb_load_all_3yr_filter:.3f}s, {n_keys_filtered:,} keys")
print(f"Filter ratio: {n_keys_filtered/n_keys*100:.1f}% of data")

load_all_prices (3yr filter): 1.000s, 977,163 keys
Filter ratio: 11.5% of data


In [5]:
# Benchmark load_prices_matrix (no filter)
clear_prices_matrix()
t0 = time.perf_counter()
pm = load_prices_matrix(PRICES_FILE)
t1 = time.perf_counter()
duckdb_load_matrix_no_filter = t1 - t0
print(f"load_prices_matrix (no filter): {duckdb_load_matrix_no_filter:.3f}s")
print(f"Matrix shape: {pm.price_matrix.shape} ({pm.n_rows} ticker|fields x {pm.n_cols} dates)")
print(f"Memory: {pm.price_matrix.nbytes / 1024 / 1024:.1f} MB")

load_prices_matrix (no filter): 6.212s
Matrix shape: (4850, 6649) (4850 ticker|fields x 6649 dates)
Memory: 246.0 MB


## Step 2: Profile Where Time Goes (DuckDB)

In [6]:
# Profile DuckDB query vs Python loop

# DuckDB query time
con = duckdb.connect()
t0 = time.perf_counter()
result = con.execute(f"SELECT ticker, field, date, value FROM '{PRICES_FILE}'").fetchall()
t1 = time.perf_counter()
duckdb_query_time = t1 - t0
con.close()
print(f"DuckDB query + fetchall: {duckdb_query_time:.3f}s, {len(result):,} rows")

# Python dict building time
t0 = time.perf_counter()
test_dict = {}
for ticker, field, dt, value in result:
    key = f"{ticker}|{field}|{dt}"
    test_dict[key] = str(value)
t1 = time.perf_counter()
python_dict_time = t1 - t0
print(f"Python dict building: {python_dict_time:.3f}s")

print(f"\nBreakdown:")
print(f"  DuckDB query: {duckdb_query_time/duckdb_load_all_no_filter*100:.1f}%")
print(f"  Python loop:  {python_dict_time/duckdb_load_all_no_filter*100:.1f}%")

DuckDB query + fetchall: 2.604s, 8,489,913 rows


Python dict building: 7.178s

Breakdown:
  DuckDB query: 29.2%
  Python loop:  80.5%


## Step 3: Implement PyArrow Alternative

In [7]:
def load_all_prices_arrow(
    prices_parquet: str,
    start_date: str | None = None,
    end_date: str | None = None
) -> dict[str, str]:
    """Load prices using PyArrow (benchmark version)."""
    
    # Read parquet
    table = pq.read_table(prices_parquet)
    
    # Optional date filter - need to convert string dates to date objects for Arrow comparison
    if start_date or end_date:
        conditions = []
        if start_date:
            start_dt = date.fromisoformat(start_date)
            conditions.append(pc.greater_equal(table['date'], pa.scalar(start_dt, type=pa.date32())))
        if end_date:
            end_dt = date.fromisoformat(end_date)
            conditions.append(pc.less_equal(table['date'], pa.scalar(end_dt, type=pa.date32())))
        if len(conditions) == 2:
            mask = pc.and_(conditions[0], conditions[1])
        else:
            mask = conditions[0]
        table = table.filter(mask)
    
    # Build dict using Arrow compute (vectorized string concat)
    # Convert date to string first
    date_str = pc.strftime(table['date'], '%Y-%m-%d')
    
    # Build composite key: ticker|field|date
    keys = pc.binary_join_element_wise(
        pc.binary_join_element_wise(table['ticker'], table['field'], '|'),
        date_str,
        '|'
    )
    values = pc.cast(table['value'], pa.string())
    
    # Convert to Python dict
    return dict(zip(keys.to_pylist(), values.to_pylist()))

In [8]:
# Profile PyArrow approach - break down steps

# Step 1: Read parquet
t0 = time.perf_counter()
table = pq.read_table(PRICES_FILE)
t1 = time.perf_counter()
arrow_read_time = t1 - t0
print(f"PyArrow read_table: {arrow_read_time:.3f}s, {table.num_rows:,} rows")

# Step 2: String operations
t0 = time.perf_counter()
date_str = pc.strftime(table['date'], '%Y-%m-%d')
keys = pc.binary_join_element_wise(
    pc.binary_join_element_wise(table['ticker'], table['field'], '|'),
    date_str,
    '|'
)
values = pc.cast(table['value'], pa.string())
t1 = time.perf_counter()
arrow_compute_time = t1 - t0
print(f"Arrow compute (keys/values): {arrow_compute_time:.3f}s")

# Step 3: Convert to Python dict
t0 = time.perf_counter()
keys_list = keys.to_pylist()
values_list = values.to_pylist()
t1 = time.perf_counter()
arrow_to_python_time = t1 - t0
print(f"Arrow to Python lists: {arrow_to_python_time:.3f}s")

t0 = time.perf_counter()
result_dict = dict(zip(keys_list, values_list))
t1 = time.perf_counter()
dict_creation_time = t1 - t0
print(f"Dict creation: {dict_creation_time:.3f}s")

arrow_total = arrow_read_time + arrow_compute_time + arrow_to_python_time + dict_creation_time
print(f"\nTotal PyArrow: {arrow_total:.3f}s")

PyArrow read_table: 0.105s, 8,489,913 rows


Arrow compute (keys/values): 3.710s


Arrow to Python lists: 5.049s
Dict creation: 1.724s

Total PyArrow: 10.588s


## Step 4: Benchmark PyArrow vs DuckDB

In [9]:
# Benchmark PyArrow (no filter)
t0 = time.perf_counter()
prices_dict_arrow = load_all_prices_arrow(PRICES_FILE)
t1 = time.perf_counter()
arrow_load_all_no_filter = t1 - t0
print(f"load_all_prices_arrow (no filter): {arrow_load_all_no_filter:.3f}s, {len(prices_dict_arrow):,} keys")

load_all_prices_arrow (no filter): 10.513s, 8,489,913 keys


In [10]:
# Benchmark PyArrow (3-year filter)
t0 = time.perf_counter()
prices_dict_arrow_filtered = load_all_prices_arrow(PRICES_FILE, "2022-01-01", "2024-12-31")
t1 = time.perf_counter()
arrow_load_all_3yr_filter = t1 - t0
print(f"load_all_prices_arrow (3yr filter): {arrow_load_all_3yr_filter:.3f}s, {len(prices_dict_arrow_filtered):,} keys")

load_all_prices_arrow (3yr filter): 1.189s, 977,163 keys


In [11]:
# Verify correctness - keys should match
clear_prices_dict()
prices_dict_duckdb = load_all_prices(PRICES_FILE)

# Compare key sets
duckdb_keys = set(prices_dict_duckdb.keys())
arrow_keys = set(prices_dict_arrow.keys())

print(f"DuckDB keys: {len(duckdb_keys):,}")
print(f"Arrow keys:  {len(arrow_keys):,}")
print(f"Keys match:  {duckdb_keys == arrow_keys}")

if duckdb_keys != arrow_keys:
    print(f"Only in DuckDB: {len(duckdb_keys - arrow_keys)}")
    print(f"Only in Arrow: {len(arrow_keys - duckdb_keys)}")
    # Show sample differences
    if duckdb_keys - arrow_keys:
        print(f"Sample DuckDB-only: {list(duckdb_keys - arrow_keys)[:3]}")
    if arrow_keys - duckdb_keys:
        print(f"Sample Arrow-only: {list(arrow_keys - duckdb_keys)[:3]}")

DuckDB keys: 8,489,913
Arrow keys:  8,489,913
Keys match:  True


In [12]:
# Compare values for matching keys
if duckdb_keys == arrow_keys:
    mismatches = 0
    for key in list(duckdb_keys)[:10000]:  # Sample first 10k
        if prices_dict_duckdb[key] != prices_dict_arrow[key]:
            mismatches += 1
            if mismatches <= 3:
                print(f"Mismatch: {key}")
                print(f"  DuckDB: {prices_dict_duckdb[key]}")
                print(f"  Arrow:  {prices_dict_arrow[key]}")
    print(f"Value mismatches in first 10k: {mismatches}")

Mismatch: LAQ2007 Comdty|PX_LAST|2003-08-01
  DuckDB: 1431.0
  Arrow:  1431
Mismatch: LPH2006 Comdty|PX_LAST|2004-07-13
  DuckDB: 2356.0
  Arrow:  2356
Mismatch: LPK2023 Comdty|PX_LAST|2015-04-22
  DuckDB: 6012.0
  Arrow:  6012
Value mismatches in first 10k: 967


## Step 5: PriceMatrix Alternative (PyArrow)

In [13]:
from specparser.amt.prices import PriceMatrix

_EPOCH = date(1970, 1, 1)

def load_prices_matrix_arrow(
    prices_parquet: str,
    start_date: str | None = None,
    end_date: str | None = None,
) -> PriceMatrix:
    """Load prices into PriceMatrix using PyArrow."""
    
    # Read parquet
    table = pq.read_table(prices_parquet)
    
    # Optional date filter - need to convert string dates to date objects for Arrow comparison
    if start_date or end_date:
        conditions = []
        if start_date:
            start_dt = date.fromisoformat(start_date)
            conditions.append(pc.greater_equal(table['date'], pa.scalar(start_dt, type=pa.date32())))
        if end_date:
            end_dt = date.fromisoformat(end_date)
            conditions.append(pc.less_equal(table['date'], pa.scalar(end_dt, type=pa.date32())))
        if len(conditions) == 2:
            mask = pc.and_(conditions[0], conditions[1])
        else:
            mask = conditions[0]
        table = table.filter(mask)
    
    if table.num_rows == 0:
        return PriceMatrix(
            price_matrix=np.empty((0, 0), dtype=np.float64),
            ticker_field_to_row={},
            date32_to_col=np.array([], dtype=np.int32),
            min_date32=0,
        )
    
    # Build ticker|field key column
    ticker_field = pc.binary_join_element_wise(table['ticker'], table['field'], '|')
    
    # Get unique ticker|fields and dates
    unique_tf = pc.unique(ticker_field).to_pylist()
    unique_tf_sorted = sorted(unique_tf)
    ticker_field_to_row = {tf: i for i, tf in enumerate(unique_tf_sorted)}
    n_rows = len(unique_tf_sorted)
    
    # Convert dates to date32 (days since epoch)
    epoch_ordinal = _EPOCH.toordinal()
    dates_py = table['date'].to_pylist()
    date32_values = np.array([d.toordinal() - epoch_ordinal for d in dates_py], dtype=np.int32)
    
    unique_date32 = sorted(set(date32_values))
    date32_to_col_dict = {d: i for i, d in enumerate(unique_date32)}
    n_cols = len(unique_date32)
    
    min_date32 = unique_date32[0]
    max_date32 = unique_date32[-1]
    n_calendar_days = max_date32 - min_date32 + 1
    
    # Build dense date32_to_col array
    date32_to_col = np.full(n_calendar_days, -1, dtype=np.int32)
    for d32, col in date32_to_col_dict.items():
        date32_to_col[d32 - min_date32] = col
    
    # Build row indices for each entry
    tf_list = ticker_field.to_pylist()
    row_indices = np.array([ticker_field_to_row[tf] for tf in tf_list], dtype=np.int32)
    col_indices = np.array([date32_to_col_dict[d32] for d32 in date32_values], dtype=np.int32)
    
    # Get values as float array
    values = table['value'].to_numpy().astype(np.float64)
    
    # Fill price matrix using numpy advanced indexing (vectorized!)
    price_matrix = np.full((n_rows, n_cols), np.nan, dtype=np.float64)
    price_matrix[row_indices, col_indices] = values
    
    return PriceMatrix(
        price_matrix=price_matrix,
        ticker_field_to_row=ticker_field_to_row,
        date32_to_col=date32_to_col,
        min_date32=min_date32,
    )

In [14]:
# Benchmark load_prices_matrix_arrow (no filter)
t0 = time.perf_counter()
pm_arrow = load_prices_matrix_arrow(PRICES_FILE)
t1 = time.perf_counter()
arrow_load_matrix_no_filter = t1 - t0
print(f"load_prices_matrix_arrow (no filter): {arrow_load_matrix_no_filter:.3f}s")
print(f"Matrix shape: {pm_arrow.price_matrix.shape}")

load_prices_matrix_arrow (no filter): 8.600s
Matrix shape: (4850, 6649)


In [15]:
# Verify PriceMatrix correctness
clear_prices_matrix()
pm_duckdb = load_prices_matrix(PRICES_FILE)

print(f"DuckDB matrix shape: {pm_duckdb.price_matrix.shape}")
print(f"Arrow matrix shape:  {pm_arrow.price_matrix.shape}")
print(f"Shapes match: {pm_duckdb.price_matrix.shape == pm_arrow.price_matrix.shape}")

if pm_duckdb.price_matrix.shape == pm_arrow.price_matrix.shape:
    # Compare values (accounting for NaN)
    close = np.allclose(pm_duckdb.price_matrix, pm_arrow.price_matrix, equal_nan=True)
    print(f"Values match: {close}")
    
    if not close:
        diff = np.abs(pm_duckdb.price_matrix - pm_arrow.price_matrix)
        diff_mask = ~np.isnan(diff) & (diff > 1e-10)
        print(f"Number of differing values: {np.sum(diff_mask)}")

DuckDB matrix shape: (4850, 6649)
Arrow matrix shape:  (4850, 6649)
Shapes match: True
Values match: True


## Results Summary

In [16]:
import pandas as pd

results = {
    'Function': [
        'load_all_prices (no filter)',
        'load_all_prices (3yr filter)',
        'load_prices_matrix (no filter)',
    ],
    'DuckDB (s)': [
        duckdb_load_all_no_filter,
        duckdb_load_all_3yr_filter,
        duckdb_load_matrix_no_filter,
    ],
    'PyArrow (s)': [
        arrow_load_all_no_filter,
        arrow_load_all_3yr_filter,
        arrow_load_matrix_no_filter,
    ],
}

df = pd.DataFrame(results)
df['Speedup'] = df['DuckDB (s)'] / df['PyArrow (s)']
df['Winner'] = df.apply(lambda r: 'PyArrow' if r['Speedup'] > 1 else 'DuckDB', axis=1)

print("\n" + "="*70)
print("BENCHMARK RESULTS")
print("="*70)
print(df.to_string(index=False))
print("="*70)


BENCHMARK RESULTS
                      Function  DuckDB (s)  PyArrow (s)  Speedup Winner
   load_all_prices (no filter)    8.921209    10.512742 0.848609 DuckDB
  load_all_prices (3yr filter)    1.000249     1.188990 0.841259 DuckDB
load_prices_matrix (no filter)    6.211921     8.599905 0.722324 DuckDB


In [17]:
# Visual comparison (requires matplotlib)
try:
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(figsize=(10, 5))

    x = np.arange(len(df))
    width = 0.35

    bars1 = ax.bar(x - width/2, df['DuckDB (s)'], width, label='DuckDB', color='steelblue')
    bars2 = ax.bar(x + width/2, df['PyArrow (s)'], width, label='PyArrow', color='coral')

    ax.set_ylabel('Time (seconds)')
    ax.set_title('Price Loading: DuckDB vs PyArrow')
    ax.set_xticks(x)
    ax.set_xticklabels(['load_all_prices\n(no filter)', 'load_all_prices\n(3yr filter)', 'load_prices_matrix\n(no filter)'])
    ax.legend()

    # Add speedup annotations
    for i, (b1, b2, speedup) in enumerate(zip(bars1, bars2, df['Speedup'])):
        max_height = max(b1.get_height(), b2.get_height())
        ax.annotate(f'{speedup:.2f}x', 
                    xy=(i, max_height + 0.1),
                    ha='center', va='bottom',
                    fontsize=10, fontweight='bold')

    plt.tight_layout()
    plt.show()
except ImportError:
    print("(matplotlib not available - skipping chart)")

(matplotlib not available - skipping chart)


## Analysis & Recommendation

In [18]:
print("\n" + "="*70)
print("ANALYSIS")
print("="*70)

avg_speedup = df['Speedup'].mean()
print(f"\nAverage speedup: {avg_speedup:.2f}x")

print("\nBreakdown for load_all_prices (no filter):")
print(f"  DuckDB query + fetchall: {duckdb_query_time:.3f}s ({duckdb_query_time/duckdb_load_all_no_filter*100:.1f}%)")
print(f"  Python dict building:    {python_dict_time:.3f}s ({python_dict_time/duckdb_load_all_no_filter*100:.1f}%)")

print(f"\nPyArrow breakdown:")
print(f"  read_table:     {arrow_read_time:.3f}s")
print(f"  Arrow compute:  {arrow_compute_time:.3f}s")
print(f"  to_pylist:      {arrow_to_python_time:.3f}s")
print(f"  dict(zip):      {dict_creation_time:.3f}s")

print("\n" + "="*70)
print("RECOMMENDATION")
print("="*70)

if avg_speedup >= 1.2:
    print(f"\n✅ SWITCH TO PYARROW")
    print(f"   Average speedup of {avg_speedup:.2f}x justifies the change.")
elif avg_speedup >= 1.0:
    print(f"\n⚠️  MARGINAL IMPROVEMENT")
    print(f"   Average speedup of {avg_speedup:.2f}x - consider if worth the code change.")
else:
    print(f"\n❌ KEEP DUCKDB")
    print(f"   PyArrow is slower (speedup {avg_speedup:.2f}x).")

print("\nNotes:")
print("- These results may vary based on file size and data characteristics")
print("- DuckDB has better predicate pushdown for selective queries")
print("- PyArrow is better when reading most/all of the data")


ANALYSIS

Average speedup: 0.80x

Breakdown for load_all_prices (no filter):
  DuckDB query + fetchall: 2.604s (29.2%)
  Python dict building:    7.178s (80.5%)

PyArrow breakdown:
  read_table:     0.105s
  Arrow compute:  3.710s
  to_pylist:      5.049s
  dict(zip):      1.724s

RECOMMENDATION

❌ KEEP DUCKDB
   PyArrow is slower (speedup 0.80x).

Notes:
- These results may vary based on file size and data characteristics
- DuckDB has better predicate pushdown for selective queries
- PyArrow is better when reading most/all of the data


## Memory Comparison

In [19]:
import sys

# Approximate memory usage
duckdb_dict_size = sys.getsizeof(prices_dict_duckdb)
arrow_dict_size = sys.getsizeof(prices_dict_arrow)

# For dicts, need to also count keys and values
duckdb_total = duckdb_dict_size + sum(sys.getsizeof(k) + sys.getsizeof(v) for k, v in list(prices_dict_duckdb.items())[:1000]) / 1000 * len(prices_dict_duckdb)
arrow_total = arrow_dict_size + sum(sys.getsizeof(k) + sys.getsizeof(v) for k, v in list(prices_dict_arrow.items())[:1000]) / 1000 * len(prices_dict_arrow)

print(f"Estimated dict memory usage:")
print(f"  DuckDB result: ~{duckdb_total/1024/1024:.1f} MB")
print(f"  Arrow result:  ~{arrow_total/1024/1024:.1f} MB")

print(f"\nPriceMatrix memory:")
print(f"  Matrix array: {pm_duckdb.price_matrix.nbytes/1024/1024:.1f} MB")

Estimated dict memory usage:
  DuckDB result: ~1350.7 MB
  Arrow result:  ~1350.7 MB

PriceMatrix memory:
  Matrix array: 246.0 MB
